Reference
https://python.langchain.com/docs/tutorials/rag/    

#### Configuration

1. VS Code > Kernel > Install Python + Jupyter   

2. Create a virtual environment for Python 

3. Install tools    

In [2]:
pip install --upgrade pip

     |████████████████████████████████| 1.8 MB 7.1 MB/s eta 0:00:01
  Attempting uninstall: pip
    Found existing installation: pip 21.2.4
    Uninstalling pip-21.2.4:
      Successfully uninstalled pip-21.2.4
Note: you may need to restart the kernel to use updated packages.


#### Install LangChain & Tools

In [ ]:
pip install --quiet --upgrade langchain-text-splitters langchain-community langgraph

Note: you may need to restart the kernel to use updated packages.


In [1]:
pip install -qU "langchain[openai]"

Note: you may need to restart the kernel to use updated packages.


##### Store an open key - env
https://abc-notes.data.tech.gov.sg/notes/topic-6-ai-agents-with-tools/2.-a-more-secure-way-to-store-credentials.html

In [ ]:
pip install python-dotenv

  Using cached python_dotenv-1.1.0-py3-none-any.whl.metadata (24 kB)
Using cached python_dotenv-1.1.0-py3-none-any.whl (20 kB)
Note: you may need to restart the kernel to use updated packages.


In [14]:
pip install -qU langchain-openai

Note: you may need to restart the kernel to use updated packages.


##### Vector DB - Elasticsearch    

* Tutorial : https://python.langchain.com/docs/integrations/vectorstores/elasticsearch/    

* Blog : https://www.elastic.co/search-labs/blog/dataset-translation-langchain-python-elastic#:~:text=Loading%20the%20translated%20articles%20into%20a%20vector%20database%20and%20searching

##### Elastic Cloud

* Web : https://cloud.elastic.co

* Free-trial : https://cloud.elastic.co/registration?utm_source=langchain&utm_content=documentation    

* Get started with Elasticsearch Serverless : 
https://www.elastic.co/docs/solutions/search/serverless-elasticsearch-get-started

In [15]:
pip install langchain-elasticsearch

Note: you may need to restart the kernel to use updated packages.


In [1]:
%pip install pdf2image -q
%pip install pdfminer -q
%pip install pdfminer.six -q
%pip install openai -q
%pip install scikit-learn -q
%pip install rich -q
%pip install tqdm -q
%pip install pandas -q

Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.


# Install poppler in the terminal
brew install poppler

#### code

In [ ]:
# Imports
from pdf2image import convert_from_path
from pdf2image.exceptions import (
    PDFInfoNotInstalledError,
    PDFPageCountError,
    PDFSyntaxError
)

from pdfminer.high_level import extract_text
import base64
import io
import os
from openai import OpenAI
import re 
import numpy as np
from rich import print

def convert_doc_to_images(path):
    images = convert_from_path(path)
    return images

def extract_text_from_doc(path):
    text = extract_text(path)
    return text

In [9]:
%pip install pillow -q

Note: you may need to restart the kernel to use updated packages.


#### Convert pdf to image
https://cookbook.openai.com/examples/parse_pdf_docs_for_rag


In [ ]:
file_path = os.path.abspath("data.pdf")
imgs = convert_doc_to_images(file_path)
for img in imgs:
    display(img)

In [ ]:
import os  
from dotenv import load_dotenv
from langchain.chat_models import init_chat_model
from langchain_openai import OpenAIEmbeddings
from langchain_elasticsearch import ElasticsearchStore
from langchain.schema import Document
from langchain.schema.output_parser import StrOutputParser

In [182]:
load_dotenv()

True

In [ ]:
# Converting images to base64 encoded images
def get_img_uri(img):
    png_buffer = io.BytesIO()
    img.save(png_buffer, format="PNG")
    png_buffer.seek(0)

    base64_png = base64.b64encode(png_buffer.read()).decode('utf-8')

    data_uri = f"data:image/png;base64,{base64_png}"
    return data_uri

#### Extract text from image

In [138]:
# PDF - 설명글 / 테이블 / 다이어그램 / 코드 / 참고문헌

extract_prompt = '''
You will be provided with images containing text, representing multiple pages of a document.

Your task is to extract all readable text from the images **as accurately and completely as possible, and organize it by logical sections.

- **Do Not** add any explanations, summaries, or interpretations.
- **Do Not** mark or mention page numbers in the output.
- Remove any isolated date stamps, page numbers, headers, footer, or other non-content elements.
- Exclude any repeated headers titled "Prompt Engineering"
- If a subheading is clearly present (e.g., bolded, underlined, capitalized, or formatted distinctly), start a new section from that point.
- Maintain the original order, flow of the content, and line breaks.

- **Diagrams**: Explain each component and how they interact. For example, "The process begins with X, which then leads to Y and results in Z."
- **Tables**: Break down the information logically. In the table, bolded characters represent the type or name of the data (such as variable names or categories), while non-bolded characters represent the actual data values. For example, "Product A costs 100 dollars, while Product B is priced at 200 dollars."

Output should be structured as a sequence of content sections divided by meaningful subheadings or topic shifts.

------

If there is an identifiable title, present the output in the following format:

{TITLE}
{Content description}
'''

client = OpenAI()

def analyze_image(data_uri):
    response = client.chat.completions.create(
        model="gpt-4o",
        messages=[
            {"role": "system", "content": extract_prompt},
            {
                "role": "user",
                "content": [
                    {
                        "type": "image_url",
                        "image_url": {"url": f"{data_uri}"}
                    }
                ]
            },
        ],
        max_tokens=500,
        temperature=0,
        top_p=0.1
    )
    return response.choices[0].message.content

In [74]:
# 텍스트 정리
def clean_text(text):
    content = text.replace(' \n', '').replace('\n\n', '\n').replace('\n\n\n', '\n').strip()
    content = re.sub(r"\*{1,2}", "", content)
    return content

In [139]:
# 텍스트 추출
documents = []
for i, img in enumerate(imgs):
    data_uri = get_img_uri(img)  # img 객체를 data_uri로 변환
    text = analyze_image(data_uri)
    text = clean_text(text)
    doc = Document(page_content=text, metadata={"image_name": f"page_{i+1}"})
    documents.append(doc)

In [ ]:
for doc in documents:
    print(doc)

#### Data cleaning

In [ ]:
doc = documents[11]
print(doc)

new_content = doc.page_content.replace('-', '')
new = Document(metadata=doc.metadata, page_content=new_content)
print(new)

Document(
    metadata={'image_name': 'page_12'},
    page_content='System, contextual and role prompting\nSystem, contextual and role prompting are all techniques 
used to guide how LLMs generate text, but they focus on different aspects:\n- System prompting sets the overall 
context and purpose for the language model. It defines the ‘big picture’ of what the model should be doing, like 
translating a language, classifying a review etc.\n- Contextual prompting provides specific details or background 
information relevant to the current conversation or task. It helps the model to understand the nuances of what’s 
being asked and tailor the response accordingly.\n- Role prompting assigns a specific character or identity for the
language model to adopt. This helps the model generate responses that are consistent with the assigned role and its
associated knowledge and behavior.\nThere can be considerable overlap between system, contextual, and role 
prompting. E.g. a prompt that assigns a role to the system, can also have a context.\nHowever, each type of prompt 
serves a slightly different primary purpose:\n- System prompt: Defines the model’s fundamental capabilities and 
overarching purpose.\n- Contextual prompt: Provides immediate, task-specific information to guide the response. 
It’s highly specific to the current task or input, which is dynamic.\n- Role prompt: Frames the model’s output 
style and voice. It adds a layer of specificity and personality.'
)

Document(
    metadata={'image_name': 'page_12'},
    page_content='System, contextual and role prompting\nSystem, contextual and role prompting are all techniques 
used to guide how LLMs generate text, but they focus on different aspects:\n System prompting sets the overall 
context and purpose for the language model. It defines the ‘big picture’ of what the model should be doing, like 
translating a language, classifying a review etc.\n Contextual prompting provides specific details or background 
information relevant to the current conversation or task. It helps the model to understand the nuances of what’s 
being asked and tailor the response accordingly.\n Role prompting assigns a specific character or identity for the 
language model to adopt. This helps the model generate responses that are consistent with the assigned role and its
associated knowledge and behavior.\nThere can be considerable overlap between system, contextual, and role 
prompting. E.g. a prompt that assigns a role to the system, can also have a context.\nHowever, each type of prompt 
serves a slightly different primary purpose:\n System prompt: Defines the model’s fundamental capabilities and 
overarching purpose.\n Contextual prompt: Provides immediate, taskspecific information to guide the response. It’s 
highly specific to the current task or input, which is dynamic.\n Role prompt: Frames the model’s output style and 
voice. It adds a layer of specificity and personality.'
)

In [ ]:
documents[11] = new
print(documents[11])

In [149]:
doc = documents[12]
print(doc)

new_content = re.sub(r'\s{2,}', ' ', doc.page_content.replace('-', ''))
new = Document(metadata=doc.metadata, page_content=new_content)
print(new)

Document(
    metadata={'image_name': 'page_13'},
    page_content='Distinguishing between system, contextual, and role prompts provides a framework for designing 
prompts with clear intent, allowing for flexible combinations and making it easier to analyze how each prompt type 
influences the language model’s output.\nLet’s dive into these three different kinds of prompts.\nSystem 
prompting\nTable 3 contains a system prompt, where I specify additional information on how to return the output. I 
increased the temperature to get a higher creativity level, and I specified a higher token limit. However, because 
of my clear instruction on how to return the output the model didn’t return extra text.\nTable 3. An example of 
system prompting\nGoal: Classify movie reviews as positive, neutral or negative.\nModel: gemini-pro\nTemperature: 
1\nToken Limit: 5\nTop-K: 40\nTop-P: 0.8\nPrompt:  Classify movie reviews as positive, neutral or negative. Only 
return the label in uppercase.\n  Review: "Her" is a disturbing study revealing the direction humanity is headed if
AI is allowed to keep evolving, unchecked. It\'s so disturbing I couldn\'t watch it.\n  Sentiment:\nOutput: 
NEGATIVE'
)

Document(
    metadata={'image_name': 'page_13'},
    page_content='Distinguishing between system, contextual, and role prompts provides a framework for designing 
prompts with clear intent, allowing for flexible combinations and making it easier to analyze how each prompt type 
influences the language model’s output.\nLet’s dive into these three different kinds of prompts.\nSystem 
prompting\nTable 3 contains a system prompt, where I specify additional information on how to return the output. I 
increased the temperature to get a higher creativity level, and I specified a higher token limit. However, because 
of my clear instruction on how to return the output the model didn’t return extra text.\nTable 3. An example of 
system prompting\nGoal: Classify movie reviews as positive, neutral or negative.\nModel: geminipro\nTemperature: 
1\nToken Limit: 5\nTopK: 40\nTopP: 0.8\nPrompt: Classify movie reviews as positive, neutral or negative. Only 
return the label in uppercase. Review: "Her" is a disturbing study revealing the direction humanity is headed if AI
is allowed to keep evolving, unchecked. It\'s so disturbing I couldn\'t watch it. Sentiment:\nOutput: NEGATIVE'
)

In [ ]:
documents[12] = new
print(documents[12])

In [ ]:
doc = documents[14]
print(doc)

new_content = re.sub(r'\s{2,}', ' ', doc.page_content.replace('```', ''))
new = Document(metadata=doc.metadata, page_content=new_content)

print(new)

Document(
    metadata={'image_name': 'page_15'},
    page_content='Classify Movie Reviews as Positive, Neutral, or Negative, Return JSON\nGoal Classify movie 
reviews as positive, neutral, or negative, return JSON.\nModel gemini-pro\nTemperature 1\nToken Limit 1024\nTop-K 
40\nTop-P 0.8\nPrompt Classify movie reviews as positive, neutral or negative. Return valid JSON:\nReview: "Her" is
a disturbing study revealing the direction humanity is headed if AI is allowed to keep evolving, unchecked. It\'s 
so disturbing I couldn\'t watch it.\nSchema:\n```\nMOVIE:\n{\n  "sentiment": String "POSITIVE" | "NEGATIVE" | 
"NEUTRAL",\n  "name": String\n}\nMOVIE REVIEWS:\n{\n  "movie_reviews": [MOVIE]\n}\n```\nJSON 
Response:\nOutput\n```\n{\n  "movie_reviews": [\n    {\n      "sentiment": "NEGATIVE",\n      "name": "Her"\n    
}\n  ]\n}\n```\nTable 4. An example of system prompting with JSON format'
)

Document(
    metadata={'image_name': 'page_15'},
    page_content='Classify Movie Reviews as Positive, Neutral, or Negative, Return JSON\nGoal Classify movie 
reviews as positive, neutral, or negative, return JSON.\nModel gemini-pro\nTemperature 1\nToken Limit 1024\nTop-K 
40\nTop-P 0.8\nPrompt Classify movie reviews as positive, neutral or negative. Return valid JSON:\nReview: "Her" is
a disturbing study revealing the direction humanity is headed if AI is allowed to keep evolving, unchecked. It\'s 
so disturbing I couldn\'t watch it.\nSchema: MOVIE:\n{ "sentiment": String "POSITIVE" | "NEGATIVE" | "NEUTRAL", 
"name": String\n}\nMOVIE REVIEWS:\n{ "movie_reviews": [MOVIE]\n} JSON Response:\nOutput { "movie_reviews": [ { 
"sentiment": "NEGATIVE", "name": "Her" } ]\n} Table 4. An example of system prompting with JSON format'
)

In [ ]:
documents[14] = new
print(documents[14])

In [156]:
doc = documents[18]
print(doc)

new_content = re.sub(r'\s{2,}', ' ', doc.page_content.replace('-', ''))
new = Document(metadata=doc.metadata, page_content=new_content)

print(new)

Document(
    metadata={'image_name': 'page_19'},
    page_content="Suggesting Articles for a Blog About Retro Games\nGoal: Suggest articles for a blog about retro 
games\nModel: gemini-pro\nTemperature: 1\nToken Limit: 1024\nTop-K: 40\nTop-P: 0.8\nPrompt:Context: You are writing
for a blog about retro 80's arcade video games. Suggest 3 topics to write an article about with a few lines of 
description of what this article should contain.\nOutput:\n1. The Evolution of Arcade Cabinet Design:\n   - This 
article would explore the evolution of arcade cabinet designs, from the early wood and metal cabinets of the 1970s 
to the sleek, neon-lit designs of the 1980s and beyond.\n2. Blast From The Past: Iconic Arcade Games of The 80’s:\n
- This article would feature a glimpse into the history, impact, and legacy of some of the most iconic and 
influential arcade games of the 1980s, highlighting their innovations, popular mechanics, and enduring charm.\n3. 
The Rise and Retro Revival of Pixel Art:\n   - This article would delve into the evolution of pixel art as a 
defining visual style of early arcade games, tracing its roots, exploring its techniques, and discussing the 
resurgence of pixel art in modern games and digital art."
)

Document(
    metadata={'image_name': 'page_19'},
    page_content="Suggesting Articles for a Blog About Retro Games\nGoal: Suggest articles for a blog about retro 
games\nModel: geminipro\nTemperature: 1\nToken Limit: 1024\nTopK: 40\nTopP: 0.8\nPrompt:Context: You are writing 
for a blog about retro 80's arcade video games. Suggest 3 topics to write an article about with a few lines of 
description of what this article should contain.\nOutput:\n1. The Evolution of Arcade Cabinet Design: This article 
would explore the evolution of arcade cabinet designs, from the early wood and metal cabinets of the 1970s to the 
sleek, neonlit designs of the 1980s and beyond.\n2. Blast From The Past: Iconic Arcade Games of The 80’s: This 
article would feature a glimpse into the history, impact, and legacy of some of the most iconic and influential 
arcade games of the 1980s, highlighting their innovations, popular mechanics, and enduring charm.\n3. The Rise and 
Retro Revival of Pixel Art: This article would delve into the evolution of pixel art as a defining visual style of 
early arcade games, tracing its roots, exploring its techniques, and discussing the resurgence of pixel art in 
modern games and digital art."
)

In [ ]:
documents[18] = new
print(documents[18])

In [159]:
doc = documents[27]
print(doc)

new_content = re.sub(r'\s{2,}', ' ', doc.page_content.replace('---', ''))
new = Document(metadata=doc.metadata, page_content=new_content)

print(new)

Document(
    metadata={'image_name': 'page_28'},
    page_content="EMAIL:\nHi,\nI have seen you use Wordpress for your website. A great open source content 
management system. I have used it in the past too. It comes with lots of great user plugins. And it's pretty easy 
to set up.\nI did notice a bug in the contact form, which happens when you select the name field. See the attached 
screenshot of me entering text in the name field. Notice the JavaScript alert box that I inv0k3d.\nBut for the rest
it's a great website. I enjoy reading it. Feel free to leave the bug in the website, because it gives me more 
interesting things to read.\nCheers,\nHarry the Hacker.\n---\nClassify the above email as IMPORTANT or NOT 
IMPORTANT. Let's think step by step and explain why."
)

Document(
    metadata={'image_name': 'page_28'},
    page_content="EMAIL:\nHi,\nI have seen you use Wordpress for your website. A great open source content 
management system. I have used it in the past too. It comes with lots of great user plugins. And it's pretty easy 
to set up.\nI did notice a bug in the contact form, which happens when you select the name field. See the attached 
screenshot of me entering text in the name field. Notice the JavaScript alert box that I inv0k3d.\nBut for the rest
it's a great website. I enjoy reading it. Feel free to leave the bug in the website, because it gives me more 
interesting things to read.\nCheers,\nHarry the Hacker. Classify the above email as IMPORTANT or NOT IMPORTANT. 
Let's think step by step and explain why."
)

In [ ]:
documents[27] = new
print(documents[27])

In [166]:
doc = documents[57]
print(doc)

new_content = re.sub(r'[-|｜]', ' ', doc.page_content)
new_content = re.sub(r'\s{2,}', ' ', new_content)
new = Document(metadata=doc.metadata, page_content=new_content)

print(new)

Document(
    metadata={'image_name': 'page_58'},
    page_content='Once you feel the prompt is close to perfect, take it to your project codebase. And in the 
codebase, save prompts in a separate file from code, so it’s easier to maintain. Finally, ideally your prompts are 
part of an operationalized system, and as a prompt engineer you should rely on automated tests and evaluation 
procedures to understand how well your prompt generalizes to a task.\nPrompt engineering is an iterative process. 
Craft and test different prompts, analyze, and document the results. Refine your prompt based on the model’s 
performance. Keep experimenting until you achieve the desired output. When you change a model or model 
configuration, go back and keep experimenting with the previously used prompts.\nTable 21. A template for 
documenting prompts\n| Name          | [name and version of your prompt]            
|\n|-------------------|----------------------------------------------|\n| Goal          | [One sentence 
explanation of the goal of this attempt] |\n| Model         | [name and version of the used model]         |\n| 
Temperature   | [value between 0 - 1]                        |\n| Token Limit   | [number]                         
|\n| Top-K         | [number]                                     |\n| Top-P         | [number]                    
|\n| Prompt        | [Write all the full prompt]                  |\n| Output        | [Write out the output or 
multiple outputs]   |\nSummary\nThis whitepaper discusses prompt engineering. We learned various prompting 
techniques, such as:\n- Zero prompting\n- Few shot prompting'
)

Document(
    metadata={'image_name': 'page_58'},
    page_content='Once you feel the prompt is close to perfect, take it to your project codebase. And in the 
codebase, save prompts in a separate file from code, so it’s easier to maintain. Finally, ideally your prompts are 
part of an operationalized system, and as a prompt engineer you should rely on automated tests and evaluation 
procedures to understand how well your prompt generalizes to a task.\nPrompt engineering is an iterative process. 
Craft and test different prompts, analyze, and document the results. Refine your prompt based on the model’s 
performance. Keep experimenting until you achieve the desired output. When you change a model or model 
configuration, go back and keep experimenting with the previously used prompts.\nTable 21. A template for 
documenting prompts Name [name and version of your prompt] Goal [One sentence explanation of the goal of this 
attempt] Model [name and version of the used model] Temperature [value between 0 1] Token Limit [number] Top K 
[number] Top P [number] Prompt [Write all the full prompt] Output [Write out the output or multiple outputs] 
Summary\nThis whitepaper discusses prompt engineering. We learned various prompting techniques, such as: Zero 
prompting Few shot prompting'
)

In [ ]:
documents[57] = new
print(documents[57])

In [169]:
doc = documents[58]
print(doc)

new_content = re.sub(r'\s{2,}', ' ', doc.page_content.replace('-', ''))
new = Document(metadata=doc.metadata, page_content=new_content)

print(new)

Document(
    metadata={'image_name': 'page_59'},
    page_content='Prompt Engineering\n- System prompting\n- Role prompting\n- Contextual prompting\n- Step-back 
prompting\n- Chain of thought\n- Self consistency\n- Tree of thoughts\n- ReAct\nWe even looked into ways how you 
can automate your prompts.\nThe whitepaper then discusses the challenges of gen AI like the problems that can 
happen when your prompts are insufficient. We closed with best practices on how to become a better prompt 
engineer.'
)

Document(
    metadata={'image_name': 'page_59'},
    page_content='Prompt Engineering System prompting Role prompting Contextual prompting Stepback prompting Chain 
of thought Self consistency Tree of thoughts ReAct\nWe even looked into ways how you can automate your 
prompts.\nThe whitepaper then discusses the challenges of gen AI like the problems that can happen when your 
prompts are insufficient. We closed with best practices on how to become a better prompt engineer.'
)

In [ ]:
documents[58] = new
print(documents[58])

#### Elasticsearch

In [183]:
# elasticsearch 연결 (~05/28 Free-Trial)
from elasticsearch import Elasticsearch

es = Elasticsearch(
    hosts=os.getenv("ELASTICSEARCH_URL"),
    api_key=os.getenv("ELASTIC_API_KEY")
)

In [ ]:
print(es.info())

#### Vector Embedding

In [62]:
embeddings = OpenAIEmbeddings(model="text-embedding-3-small")

In [174]:
# document + 임베딩 저장
es_store = ElasticsearchStore.from_documents(
    documents=documents,
    es_connection=es,
    index_name="llm-elastic",
    embedding=embeddings
)

In [185]:
# 검색 테스트
results = es_store.similarity_search("What is top k?", k=1)
for r in results:
    print(r.page_content)

The Gemini temperature control can be understood in a similar way to the softmax function used in machine learning.
A low temperature setting mirrors a low softmax temperature (T), emphasizing a single, preferred temperature with 
high certainty. A higher Gemini temperature setting is like a high softmax temperature, making a wider range of 
temperatures around the selected setting more acceptable. This increased uncertainty accommodates scenarios where a
rigid, precise temperature may not be essential like for example when experimenting with creative outputs.
Top-K and top-P
Top-K and top-P (also known as nucleus sampling) are two sampling settings used in LLMs to restrict the predicted 
next token to come from tokens with the top predicted probabilities. Like temperature, these sampling settings 
control the randomness and diversity of generated text.
- Top-K sampling selects the top K most likely tokens from the model’s predicted distribution. The higher top-K, 
the more creative and varied the model’s output; the lower top-K, the more restive and factual the model’s output. 
A top-K of 1 is equivalent to greedy decoding.
- Top-P sampling selects the top tokens whose cumulative probability does not exceed a certain value (P). Values 
for P range from 0 (greedy decoding) to 1 (all tokens in the LLM’s vocabulary).
The best way to choose between top-K and top-P is to experiment with both methods (or both together) and see which 
one produces the results you are looking for.

#### Question & Answer

In [ ]:
llm = init_chat_model("gpt-4o", model_provider="openai")

In [106]:
from langchain.prompts import ChatPromptTemplate

In [ ]:
system_prompt = '''
You will be provided with an input prompt and content as context that can be used to reply to the prompt.
    
You will do 2 things:
    
1. First, you will internally assess whether the content provided is relevant to reply to the input prompt.     
2a. If that is the case, answer directly using this content. If the content is relevant, use elements found in the content to craft a reply to the input prompt.
2b. If the content is not relevant, use your own knowledge to reply or say that you don't know how to respond if your knowledge is not sufficient to answer.
    
Stay concise with your answer, replying specifically to the input prompt without mentioning additional information provided in the context content.
'''

prompt = ChatPromptTemplate.from_messages([
    ("system", system_prompt),
    ("user", "Content:\n{content}\n\nQuestion: {question}")
])

# 벡터 검색 + GPT 답변
def answer_question(question):
    retriever = es_store.as_retriever(
        search_type="similarity_score_threshold",
        search_kwargs={"score_threshold": 0.5, "k": 3})
    
    docs = retriever.get_relevant_documents(question)
    content = "\n\n".join([doc.page_content for doc in docs])
    
    messages = {
        "content" : content,
        "question" : question
    }
    
    chain = prompt | llm | StrOutputParser()

    response = chain.invoke(messages)
    answer = f"Answer:\n{response}\n\n------\nRelated content:\n{content}"
    return answer

In [186]:
user_question = "explain about contextual prompting"
answer = answer_question(user_question)
print(answer)

Answer:
Contextual prompting provides specific details or background information pertinent to the current conversation or 
task. It assists the language model in grasping the nuances of what's being asked, enabling it to tailor its 
response appropriately. This type of prompting is highly specific to the task or input at hand, helping ensure that
responses are relevant and accurate.

------
Related content:
System, contextual and role prompting
System, contextual and role prompting are all techniques used to guide how LLMs generate text, but they focus on 
different aspects:
 System prompting sets the overall context and purpose for the language model. It defines the ‘big picture’ of what
the model should be doing, like translating a language, classifying a review etc.
 Contextual prompting provides specific details or background information relevant to the current conversation or 
task. It helps the model to understand the nuances of what’s being asked and tailor the response accordingly.
 Role prompting assigns a specific character or identity for the language model to adopt. This helps the model 
generate responses that are consistent with the assigned role and its associated knowledge and behavior.
There can be considerable overlap between system, contextual, and role prompting. E.g. a prompt that assigns a role
to the system, can also have a context.
However, each type of prompt serves a slightly different primary purpose:
 System prompt: Defines the model’s fundamental capabilities and overarching purpose.
 Contextual prompt: Provides immediate, taskspecific information to guide the response. It’s highly specific to the
current task or input, which is dynamic.
 Role prompt: Frames the model’s output style and voice. It adds a layer of specificity and personality.

Prompt Engineering System prompting Role prompting Contextual prompting Stepback prompting Chain of thought Self 
consistency Tree of thoughts ReAct
We even looked into ways how you can automate your prompts.
The whitepaper then discusses the challenges of gen AI like the problems that can happen when your prompts are 
insufficient. We closed with best practices on how to become a better prompt engineer.

Step-back prompting
Step-back prompting is a technique for improving the performance by prompting the LLM to first consider a general 
question related to the specific task at hand, and then feeding the answer to that general question into a 
subsequent prompt for the specific task. This ‘step back’ allows the LLM to activate relevant background knowledge 
and reasoning processes before attempting to solve the specific problem.
By considering the broader and underlying principles, LLMs can generate more accurate and insightful responses. 
Step-back prompting encourages LLMs to think critically and apply their knowledge in new and creative ways. It 
changes the final prompt doing the task by utilizing more knowledge in the LLM’s parameters than would otherwise 
come into play when the LLM is prompted directly.
It can help to mitigate biases in LLM responses, by focusing on general principles instead of specific details, 
step-back prompting.
Let’s have a look into these examples to understand how step-back prompting can improve the results. Let’s first 
review a traditional prompt (Table 8) before we compare it to a step back prompt (Table 9):